# Automatizando la Construction de Decks the Magic - Commander

##### Maximize the number of combos subject to a budget constraint, and the selection of 100 cards

In [1]:
import urllib, json
import pandas as pd
import numpy as np
import pyomo.environ as pyo
import pyomo.opt as po
import random

In [2]:
#READ DATABASE OF COMBOS
Spell_Book=pd.read_csv('Spell_Book _MTG_Commander.csv')
#COMBO IDENTITY
Colors = ['w,g','w', 'g']
Spell_Book=Spell_Book.loc[Spell_Book['Color Identity'].isin(Colors)]

In [3]:
#LIST OF COMBOS, In the case of selenesnaya there are 949 Combos
Combo_list=[]

#Take all the combos in the spell book and make a list
for i in range(0,len(Spell_Book)):
    #Go Row by rows, extract columns 1 to 11 (Card Columns)
    Combo=Spell_Book.iloc[[i],1:11]
    #Drop Columns NA
    Combo_2=Combo.dropna(axis='columns')
    #Create a list out the combo with no extra NAN, Fx if a combo has 2 cards is simply going to be ['sacred monk, bestiary]
    Combo_3=list(Combo_2.iloc[0])
    Combo_list.append(Combo_3)

In [4]:
#NUMBER OF COMBOS
len(Combo_list)

949

In [5]:
#LIST OF CARDS
list_of_Cards = sum(Combo_list, [])
#REMOVE ALL ThE DUPLICATES FROM MY DECK
list_of_Cards=list(dict.fromkeys(list_of_Cards))

In [6]:
#NUMBER OF CARDS POSSIBLE
len(list_of_Cards)

583

In [7]:
#CREATE A COMBO VECTOR- MATRIX of 1 and 0. Example CoOMBO 1 is (1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0) , lengh is the lengh of the cards that we are interested ( w, r, w,g)
matrix_list=[]
for i in range(0,len(Combo_list)):
    #Combo Vector = [0,0,0,0,0,0.........,0,0,0,0]
    Combo_Vector=[0]*len(list_of_Cards)
    #FX testing_Combo = ['Adarkar Valkyrie', 'Sparring Mummy', 'Phyrexian Altar'] para i=0
    testing_Combo=Combo_list[i]
    for i in testing_Combo:
        try:
    #Fx i='Adarkar Valkyrie'
    #Fx .index('Adarkar Valkyrie') me devuelve el indice esta carta dentro de mi list ade cartas y reemplazo eso en mi vector de combo
            index=list_of_Cards.index(i)
            Combo_Vector[index]=1
        except:
            pass
    matrix_list.append(Combo_Vector)
#CREATE MATRIX AND MAKE SURE THE NUMBER OF COMBOS - IS IN THE ROWS AND THE CARDS ARE THE COLUMNS   
matrix_form = np.array(matrix_list)

In [8]:
#MATRIX HAS ROWS (COMBOS),COLUMNS(POSSIBLE CARDS)
matrix_form.shape

(949, 583)

In [9]:
#ADDING THE ELEMENTS OF THE ROW - IT IS BASICALLY THE NUMBER of CARDS IN EACH COMBO ,THE SAHPE IS THE NUMBER OF COMBOS
N_Cards_Combo=matrix_form.sum(axis=1)
N_Cards_Combo.shape

(949,)

# Solving the Optimization Problem - Montecarlo Simulation

#### I am going to Optimize is the number of combos in my Deck)


trace(Combo_Matrix * (Vector of Deck) - Sum(Combo_Matrix_Row))









In [10]:
#DEFINE THE NUMBER OF SIMULATIONS
N_Simulations=100000

# lIST OF RESULTS = IS THE LIST
list_results=[]

# lIST OF ALL POSIBLE DECKS
list_Deck=[]

#THIS LIST LETS ME KNOW THE NUMBER OF COMBOS IN MY DECK
List_Count_Combos=[]

for i in range(0,N_Simulations):
    #GENERATE A RANDOM DECK, I ASSUME I CAN ONLY PICK 60 CARDS, THE REST ARE 0, AS LONG AS THE LIST OF CARDS IN THIS CASE 583
    Random_Deck=np.array(random.sample([0]*(len(list_of_Cards)-60) + [1]*60, len(list_of_Cards)))
    #I SAVE MY DECK TO THE PILE OF DECKS
    list_Deck.append(Random_Deck)
    #MULTIPLY THE MATRIX WITH THE RANDOM DECK
    result_Mult=matrix_form.dot(Random_Deck)
    #if The Difference is a 0 there is a combo present ,  IF ITS NEGATIVE THERE ARE CARDS MISSING
    Diference=result_Mult-N_Cards_Combo
    #COMBOS IN THE RUN
    List_combo_run=[]
    #GO ELEMENT BY ELEMENT IN THE RESULT
    for j in Diference:
        #IF THERE IS A COMBO PRESENT ADD A 1
        if j==0:
            List_combo_run.append(1)
        #IF THERE IS NO COMBO PRESENT ADD A 0
        else:
            List_combo_run.append(0)
    #ADD THE COMBOS PRESENT
    List_Count_Combos.append(List_combo_run)
    #MAX RESULTS IT BASICALLY SAYS IN SIMULATION # 1, THERE WHERE 1,2,3, ETC COMBIOS
    max_result=sum(List_combo_run)   
    #THE INDEX IN EACH ONE OF THIS VECTORS IS WHAT WE ARE GOING TO USE!
    list_results.append(max_result)           

In [53]:
#THIS IS OUT OF LAZYNESS--- JUST SELECT THE INDEX OF THE BEST COMBOS
df=pd.DataFrame(list(zip(list_results,List_Count_Combos,list_Deck)),columns =['#_Combos',"List_Combos","Deck_Selected"])

,#_Combos,List_Combos,Deck_Selected
0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
2,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, ..."
3,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...
99995,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
99996,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
99997,5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
99998,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [61]:
#Just use the Index to get the deck
sorted_list=df.sort_values(by='#_Combos', ascending=False)
sorted_list
#This is the Combo
#sorted_list['List_Combos'][55699])

,#_Combos,List_Combos,Deck_Selected
4193,14,"[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, ..."
55699,13,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
54705,13,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
44446,12,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
58908,12,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, ..."
...,...,...,...
35095,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
35098,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
77644,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
77643,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [64]:
sum(sorted_list['List_Combos'][55699])

13

In [11]:
#iNDEX VALUE OF THE BEST DECK
max_value = max(list_results)
max_index = list_results.index(max_value)
print('Maximum Number of Combos in this Deck is ' + str(max_value) + " and the index is " + str(max_index))

Maximum Number of Combos in this Deck is 14 and the index is 4193


In [12]:
#GET BEST DECK PRINTED
for index,card in enumerate(list_Deck[max_index]):
    if card==1:
        print(list_of_Cards[index])
    else:
        pass

Adarkar Valkyrie
Phyrexian Altar
Thornbite Staff
Altar of Dementia
Karmic Guide
Animation Module
Nyxbloom Ancient
Reveillark
Elvish Aberration
Umbral Mantle
Sunscorched Desert
Rally of Wings
Baird, Steward of Argive
Lesser Masticore
Opalescence
Life Chisel
Song of Freyalise
Quirion Ranger
Rampaging Baloths
Angelic Renewal
Triskelion
Planar Gate
Spawnsire of Ulamog
Cryptic Gateway
Sun Titan
Uba Mask
Verdant Succession
Arashin Sovereign
God-Eternal Rhonas
Wakeroot Elemental
Axis of Mortality
Abzan Battle Priest
Renata, Called to the Hunt
Painter's Servant
Archon of Falling Stars
Eldrazi Monument
Oasis Ritualist
Lightning Greaves
Dictate of Karametra
Nettle Sentinel
Oketra's Monument
Vizier of the Menagerie
Nacre Talisman
Village Bell-Ringer
Summon the School
God-Eternal Oketra
Scute Swarm
Ancient Greenwarden
Hornet Queen
Smothering Tithe
Scion Summoner
Sakura-Tribe Scout
Skyshroud Ranger
Kamahl, Heart of Krosa
Light of Promise
Puresteel Paladin
Mirari's Wake
Forsaken Monument
Sparring Mu

In [65]:
#PRINT LIST OF COMBOS IN THE BEST DECK
List_Combo_Cards=[]
for index,combo in enumerate(List_Count_Combos[max_index]):
    if combo==1:
        List_Combo_Cards.append(Combo_list[index])
        print(Combo_list[index])
    else:
        pass

["Karametra's Acolyte", 'Staff of Domination']
['Karn, the Great Creator', 'Mycosynth Lattice', 'Baird, Steward of Argive']
['Kormus Bell', 'Urborg, Tomb of Yawgmoth', 'Elesh Norn, Grand Cenobite']
['Mycosynth Lattice', 'Stony Silence', 'Baird, Steward of Argive']
['Pili-Pala', 'Mana Reflection', 'Citanul Hierophants']
['Pili-Pala', 'Mana Reflection', 'Cryptolith Rite']
['Pili-Pala', 'Nyxbloom Ancient', 'Citanul Hierophants']
['Pili-Pala', 'Nyxbloom Ancient', 'Cryptolith Rite']
['Spawnsire of Ulamog', 'Parallel Lives']
['Stony Silence', 'Mycosynth Lattice']
['Eternal Scourge', 'Food Chain']
['Emiel the Blessed', "Karametra's Acolyte", 'Lightning Greaves']
['Emiel the Blessed', "Karametra's Acolyte", 'Concordant Crossroads']


In [66]:
#important Cards
Cards_to_Get= sum(List_Combo_Cards, [])
#REMOVE ALL ThE DUPLICATES FROM MY DECK
Cards_to_Get=list(dict.fromkeys(Cards_to_Get))
Cards_to_Get 

["Karametra's Acolyte",
 'Staff of Domination',
 'Karn, the Great Creator',
 'Mycosynth Lattice',
 'Baird, Steward of Argive',
 'Kormus Bell',
 'Urborg, Tomb of Yawgmoth',
 'Elesh Norn, Grand Cenobite',
 'Stony Silence',
 'Pili-Pala',
 'Mana Reflection',
 'Citanul Hierophants',
 'Cryptolith Rite',
 'Nyxbloom Ancient',
 'Spawnsire of Ulamog',
 'Parallel Lives',
 'Eternal Scourge',
 'Food Chain',
 'Emiel the Blessed',
 'Lightning Greaves',
 'Concordant Crossroads']

In [71]:
#TEsting what is the chance to draw a combo if I draw a certain number of cards

Random_Deck=random.sample(['f']*(20) + ['w']*(20) + ['Adarkar Valkyrie',
 'Thornbite Staff',
 'Altar of Dementia',
 'Phyrexian Altar',
 'Karmic Guide',
 'Reveillark',
 'Triskelion',
 'Sun Titan',
 'Angelic Renewal',
 'Verdant Succession',
 'Arashin Sovereign',
 'God-Eternal Rhonas',
 'Renata, Called to the Hunt',
 'Lesser Masticore'] + 
                          
["Karametra's Acolyte",
 'Staff of Domination',
 'Karn, the Great Creator',
 'Mycosynth Lattice',
 'Baird, Steward of Argive',
 'Kormus Bell',
 'Urborg, Tomb of Yawgmoth',
 'Elesh Norn, Grand Cenobite',
 'Stony Silence',
 'Pili-Pala',
 'Mana Reflection',
 'Citanul Hierophants',
 'Cryptolith Rite',
 'Nyxbloom Ancient',
 'Spawnsire of Ulamog',
 'Parallel Lives',
 'Eternal Scourge',
 'Food Chain',
 'Emiel the Blessed',
 'Lightning Greaves',
 'Concordant Crossroads'] + 
    ['x']*25 , 100)

print(random.sample(Random_Deck,20))


['Mycosynth Lattice', 'x', 'w', 'x', 'w', 'f', 'f', 'Staff of Domination', 'x', 'Spawnsire of Ulamog', 'f', 'Elesh Norn, Grand Cenobite', 'Parallel Lives', 'x', 'Reveillark', 'Kormus Bell', 'Thornbite Staff', 'x', 'Cryptolith Rite', 'x']


In [ ]:
# Maximize the probability of getting a combo in my 15 draws!! That is the essence  , next step